In [27]:
import pandas as pd
x_train = pd.read_csv('data/train/subtaskA_data_all.csv')
y_train = pd.read_csv('data/train/subtaskA_answers_all.csv', header=None)
x_test = pd.read_csv('data/trial/taskA_trial_data.csv')
y_test = pd.read_csv('data/trial/taskA_trial_answer.csv', header=None)
y_train = y_train.rename(columns={0 : 'id',1: 'invalid_sent'})
y_test = y_test.rename(columns={0 : 'id',1: 'invalid_sent'})
xy = x_train.merge(y_train)
items = []
for row in xy.itertuples():
    if row.invalid_sent == 0:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 0.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 1.0})
    else:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 1.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 0.0})
df = pd.DataFrame(items)


In [28]:
xytest = x_test.merge(y_test)
items = []
for row in xytest.itertuples():
    if row.invalid_sent == 0:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 0.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 1.0})
    else:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 1.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 0.0})
testdf = pd.DataFrame(items)

In [3]:
##################### Logistic Regression #########################

In [4]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()
stop_words = set(stopwords.words('english')) 
def preprocess(x):
    x = x.lower()
    tokens = word_tokenize(x)
    filtered_tokens = [w for w in tokens if not w in stop_words] 
    return ' '.join([ps.stem(t) for t in filtered_tokens])
pdf = df.sent.apply(preprocess)
testpdf = testdf.sent.apply(preprocess)

[nltk_data] Downloading package punkt to /Users/goelprat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/goelprat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False, max_features=3000)
vectorizer.fit(pdf)
#vectorizer2 = CountVectorizer(min_df=0, lowercase=False, max_features=3000)
#vectorizer2.fit(testpdf)
len(vectorizer1.vocabulary_.keys())

3000

In [13]:
X_train = vectorizer.transform(pdf)
X_test = vectorizer.transform(testpdf)
Y_train = df.isvalid
Y_test = testdf.isvalid

In [21]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)

/Users/goelprat/miniconda3/envs/fastai/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
score = classifier.score(X_test, Y_test)
score

0.5774369124195943

In [ ]:
### Answer 5
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
### Answer 3
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(optimizer='sgd'):
    model = Sequential()
    model.add(Dense(units = 1, activation = 'sigmoid', input_dim = 3000))
    # Describe the loss and how it is optimized
    model.compile(loss = 'mean_squared_error', optimizer = optimizer, metrics = ['accuracy'])
    return model

def grid_search():
    optimizers = []
    model = KerasClassifier(build_fn=create_model)
    optimizer = ['SGD', 'Adam', 'RMSProp']
    param_grid = dict(epochs=[20, 40, 60], batch_size=[20, 40, 60], optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    grid_result = grid.fit(x_train, y_train)

    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        
model = create_model(optimizer='adam')
model.fit(X_train, Y_train, epochs=40, batch_size=40)


In [26]:
model.evaluate(X_test, Y_test)

4042/4042 [==============================] - 0s 57us/step


[0.24135216026539735, 0.5821375846862793]

In [4]:
##################### Playing with fastai library #########################

In [147]:
from fastai.text import *
class CommonSense_NN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #self.emb = nn.Embedding(vocab_size, 300) 
        self.lin1 = nn.Linear(vocab_size, 300, bias=True)
        self.lin2 = nn.Linear(300, 1, bias=True)
        
    def forward(self, x):
        #x = xb.view(-1,784) 
        #import ipdb; ipdb.set_trace()
        #x = self.emb(x)
        x = F.relu(self.lin1(x.float()))
        x = torch.sigmoid(self.lin2(x))
        return x.squeeze()

(4042, 1)

In [ ]:
from fastai.text import *
class CommonSense_Logistic(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.lin1 = nn.Linear(vocab_size, 1, bias=True)

    def forward(self, x):
        #import ipdb; ipdb.set_trace()
        x = F.sigmoid(self.lin1(x.float()))
        return x.squeeze()

In [ ]:
from fastai.text.data import TokenizeProcessor, NumericalizeProcessor
class BoWProcessor(NumericalizeProcessor):
    max_vocab = 2000
    min_freq=3
    def process_one(self,item):
        if isinstance(item, EmptyLabel): return item
        import ipdb; ipdb.set_trace()
        vsize = len(self.vocab.itos)
        res = np.zeros(vsize, dtype=np.int64)
        for x in item:
            res[self.vocab.stoi[x]] = 1
        if res is None: self.warns.append(str(item))
        return res
    
    def process(self, ds):
        if self.vocab is None: self.vocab = Vocab.create(ds.items, 2000, 3)
        ds.vocab = self.vocab
        super().process(ds)

In [ ]:
tlist = TextList.from_df(df, cols=['sent'], processor=[TokenizeProcessor(include_bos=False),BoWProcessor()])
split_data = tlist.split_by_rand_pct()
labeled_data = split_data.label_from_df(cols='isvalid')
textdb = labeled_data.databunch()
len(textdb.train_ds.x.vocab.itos)

In [ ]:
learner = Learner(data=textdb, model=CommonSense_NN(vocab_size=len(textdb.train_ds.x.vocab.itos)), loss_func=nn.MSELoss())
learner.lr_find()
learner.recorder.plot()
learner.fit_one_cycle(5,1e-06)


In [ ]:
##################### Using BERT to get document embeddings #########################

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
unk = 0
for sent in df.sent:
    sent = sent.lower()
    tokens = word_tokenize(sent)
    for token in tokens :
        if not tokenizer.vocab.get(token):
            unk += 1
print(unk)

In [46]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
bertdf = df.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
testbertdf = testdf.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tensors = []
for sent in bertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    train_tensors.append(tokens_tensor)
    
test_tensors = []
for sent in testbertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    test_tensors.append(tokens_tensor)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [60]:
train_tensors[9]

tensor([[  101,  1037, 21025, 27528,  7959,  2003,  1037,  2711,  1012,   102]])

In [61]:
tokenizer.convert_ids_to_tokens([  101,  1037, 21025, 27528,  7959,  2003,  1037,  2711,  1012,   102])

['[CLS]', 'a', 'gi', '##raf', '##fe', 'is', 'a', 'person', '.', '[SEP]']

In [48]:
import torch.nn.functional as F
import numpy as np
import math
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

X_test = np.zeros((4042, 768))
with torch.no_grad():
    for i in range(4042):
        tensor = test_tensors[i]
        outputs = model(tensor)
        embeddings = outputs[0].squeeze()
        doc = torch.mean(embeddings.squeeze(), axis=0)
        X_test[i] = doc
        
X = np.zeros((20000, 768))
with torch.no_grad():
    for i in range(20000):
        tensor = train_tensors[i]
        outputs = model(tensor)
        embeddings = outputs[0].squeeze()
        doc = torch.mean(embeddings.squeeze(), axis=0)
        X[i] = doc

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/goelprat/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://s3.amazona

In [52]:
Y = df.isvalid
Y_test = testdf.isvalid

In [53]:
Y.shape

(20000,)

In [54]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X, Y)

/Users/goelprat/miniconda3/envs/fastai/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
score = classifier.score(X_test, Y_test)
score

0.6358238495794162

In [11]:
##################### Using BERT to get perplexity #########################

In [3]:
bertdf = df.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
bertdf

0        [CLS] He poured orange juice on his cereal. [SEP]
1                [CLS] He poured milk on his cereal. [SEP]
2                             [CLS] He drinks apple. [SEP]
3                              [CLS] He drinks milk. [SEP]
4                        [CLS] Jeff ran a mile today [SEP]
                               ...                        
19995    [CLS] Harry went to the barbershop to have his...
19996          [CLS] Reilly is sleeping on the couch [SEP]
19997         [CLS] Reilly is sleeping on the window [SEP]
19998                 [CLS] I have a desk on my lamp [SEP]
19999                 [CLS] I have a lamp on my desk [SEP]
Name: sent, Length: 20000, dtype: object

In [4]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tensors = []
for sent in bertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    train_tensors.append(tokens_tensor)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [18]:
bertdf[0]

'[CLS] He poured orange juice on his cereal. [SEP]'

In [8]:
import torch.nn.functional as F
import math
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

def get_score():

# Predict all tokens
with torch.no_grad():
    for tensor in train_tensors[:2] :
        outputs = model(tensor)
        predictions = outputs[0]
        # confirm we were able to predict 'henson'
        tokens = []
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(predictions.squeeze(),tensor.squeeze()).data 
        perp = math.exp(loss)
        print(perp)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/goelprat/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://s3.amazona

35.5285578190358
40.22448497557022


In [5]:
##################### Finetuning BERT using transformers library #########################

In [3]:
bertdf = df.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
bertdf

0        [CLS] He poured orange juice on his cereal. [SEP]
1                [CLS] He poured milk on his cereal. [SEP]
2                             [CLS] He drinks apple. [SEP]
3                              [CLS] He drinks milk. [SEP]
4                        [CLS] Jeff ran a mile today [SEP]
                               ...                        
19995    [CLS] Harry went to the barbershop to have his...
19996          [CLS] Reilly is sleeping on the couch [SEP]
19997         [CLS] Reilly is sleeping on the window [SEP]
19998                 [CLS] I have a desk on my lamp [SEP]
19999                 [CLS] I have a lamp on my desk [SEP]
Name: sent, Length: 20000, dtype: object

In [7]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tensors = []
for sent in df.sent:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    train_tensors.append(tokens_tensor)

In [8]:
from torch import nn
class MyBertForSequenceClassification(nn.Module):
  
    def __init__(self, num_labels=2):
        super(MyBertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        logits = self.classifier(pooled_output)

        return logits

In [9]:
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel, BertForMaskedLM
max_seq_length = 50
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class text_dataset(Dataset):
    def __init__(self,x_y_list):
        self.x_y_list = x_y_list
        
    def __getitem__(self,index):
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])
        if len(tokenized_review) > max_seq_length:
            tokenized_review = tokenized_review[:max_seq_length]
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (max_seq_length - len(ids_review))       
        ids_review += padding      
        assert len(ids_review) == max_seq_length       
        ids_review = torch.tensor(ids_review)        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [32]:
import torch
import time
import os
import copy
import torch.nn.functional as F
import numpy as np

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            sentiment_corrects = 0
                        
            # Iterate over data.
            for inputs, targets in dataloaders_dict[phase]: 
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = F.softmax(outputs,dim=1)  
                    loss = criterion(outputs, targets.long())
                    # backward + optimize only if in training phase
                    if phase == 'train':                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                print('Running loss : {:.4f}'.format(running_loss))
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == targets.long())
            epoch_loss = running_loss / dataset_sizes[phase]
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]
            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(float(best_loss)))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [34]:
batch_size = 16
train_lists = [df.sent[:1000], df.isvalid[:1000]]
test_lists = [testdf.sent[:100], testdf.isvalid[:100]]
training_dataset = text_dataset(x_y_list = train_lists)
test_dataset = text_dataset(x_y_list = test_lists)

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [17]:
model = MyBertForSequenceClassification()

In [31]:
from torch import optim
from torch.optim import lr_scheduler
lrlast = .001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

In [ ]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

starting
Epoch 0/4
----------
Running loss : 12.3019
Running loss : 23.1574
Running loss : 34.4371
Running loss : 47.0812
Running loss : 58.0687
Running loss : 69.9240
Running loss : 79.8553
Running loss : 90.6626
Running loss : 98.7249
Running loss : 108.6554
Running loss : 121.1563
Running loss : 133.6704
Running loss : 147.1135
Running loss : 160.5545
Running loss : 173.0695
Running loss : 187.0734
Running loss : 200.6931
Running loss : 212.4535
Running loss : 225.4815
Running loss : 236.5354
Running loss : 247.6441
Running loss : 259.3732
Running loss : 270.5484
Running loss : 281.8232
Running loss : 292.8590
Running loss : 303.2048
Running loss : 314.9358
Running loss : 324.8700
Running loss : 335.8258
Running loss : 347.7164
Running loss : 359.6016
Running loss : 370.9968
Running loss : 382.9237
Running loss : 393.9334
Running loss : 404.9101
Running loss : 416.2926
Running loss : 427.3474
Running loss : 438.9927
Running loss : 450.3670
Running loss : 461.2991
Running loss : 472.

In [20]:
test_dataset

In [27]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
results = []
for inputs, targets in test_loader:
    outputs = model_ft1(inputs)
    outputs = F.softmax(outputs,dim=1)
    for i in range(len(outputs)):
        predicted_index = torch.argmax(outputs[i]).item()
        results.append(predicted_index)
    print("Loop done")

Loop done
Loop done
Loop done
Loop done
Loop done
Loop done
Loop done


In [28]:
results

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [78]:
import math
def get_score(tokens_tensor):
    predictions = None
    # Predict all tokens
    loss_fct = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
        loss = loss_fct(predictions.squeeze(),tokens_tensor.squeeze()).data 
        return math.exp(loss)

tokens_tensor = train_tensors[12]
print(get_score(tokens_tensor))
tokens_tensor = train_tensors[13]
print(get_score(tokens_tensor))

760.5078798734681
298.18919185123605


In [108]:
testdf.sent

0                      he put an elephant into the fridge
1                         he put a turkey into the fridge
2       my sister eats an apple after breakfast every day
3        my sister eats a stone after breakfast every day
4                       money can be used for buying cars
                              ...                        
4037    people have to hold onto their shoes because o...
4038    Rebecca wakes up because she takes sleeping pills
4039           Rebecca wakes up because she sets an alarm
4040                      people bleed when they feel hot
4041                      people sweat when they feel hot
Name: sent, Length: 4042, dtype: object

In [51]:
tokens_tensor.squeeze()

tensor([  101,  2002,  8542,  6501,  2006,  2010, 20943,  1012,   102])

In [128]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
transdf = testdf.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

test_tensors = []
for sent in transdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    test_tensors.append(tokens_tensor)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [136]:
transdf[3]

'[CLS] my sister eats a stone after breakfast every day [SEP]'

In [132]:
outputs = model(test_tensors[3])
predictions = outputs[0]
predictions = predictions.squeeze()
predicted_tokens = []
for i in range(len(predictions)):
    predicted_index = torch.argmax(predictions[i]).item()
    predicted_tokens.append(tokenizer.convert_ids_to_tokens([predicted_index])[0])
predicted_tokens

['.',
 '.',
 'sister',
 'eats',
 'a',
 'stone',
 'after',
 'breakfast',
 'every',
 'day',
 '.']

In [97]:
results = []
for i in range(0, len(test_tensors), 2):
    t1 = test_tensors[i]
    t2 = test_tensors[i + 1]
    s1 = get_score(t1)
    s2 = get_score(t2)
    if s1 <= s2:
        results.append(1)
    else:
        results.append(0)

KeyboardInterrupt: 

In [92]:
x_test.id

0          1
1          2
2          3
3          4
4          5
        ... 
2016    2017
2017    2018
2018    2019
2019    2020
2020    2021
Name: id, Length: 2021, dtype: int64

In [94]:
pd.concat([x_test.id, pd.Series(results)], axis=1).to_csv('./predictions.csv', header=False, index=False)

In [ ]:
!python eval/taskA_scorer.py --gold-labels data/train/subtaskA_answers_all.csv --pred-labels predictions.csv

In [95]:
!python eval/taskA_scorer.py --gold-labels data/trial/taskA_trial_answer.csv --pred-labels predictions.csv

Accuracy: 49.7279%
